In [1]:
import pandas as pd
import numpy as np

In [2]:
cc = pd.read_csv('dados/matriculas_cc.csv', sep=',')

In [3]:
PERIODO, CODIGO, DISCIPLINA, MATRICULA = 0, 1, 2, 3

network = {}
for group, obs in cc.groupby(['PERIODO', 'CODIGO']):
    
    for value_x in obs.get_values():
        mat_x = value_x[MATRICULA]
        
        for value_y in obs.get_values():
            mat_y = value_y[MATRICULA]
            
            if mat_x != mat_y:
                
                if mat_x not in network:
                    network[mat_x] = {}
                
                if mat_y not in network[mat_x]:
                    network[mat_x][mat_y] = 0
                
                network[mat_x][mat_y] += 1

In [5]:
same_num_connections = {}

for mat, connections in network.iteritems():
    
    my_connections_avg = np.mean(connections.values())
    my_connections_max = max(connections.values())
    my_n_connections = (my_connections_avg + my_connections_max) / 2
    
    my_same_n_connections= set(filter(lambda x: connections[x] >= my_n_connections and mat in connections, connections))
    same_num_connections[mat] = my_same_n_connections

In [6]:
def is_on_group(set_mother, me): 
    n_containing = len(list(filter(lambda adj: me in same_num_connections[adj], same_num_connections[set_mother])))
    return float(n_containing) / len(same_num_connections[set_mother]) >= 0.9    
    
def dfs(m):
    visited[m] = True
    my_components.append(m)
    for adj in same_num_connections[m]:
        if not visited[adj] and is_on_group(m, adj):
            dfs(adj)
            
components = []
visited = {m:False for m in network}
for mat, values in same_num_connections.iteritems():
    if not visited[mat]:
        my_components = []
        dfs(mat)
        components.append(my_components)

In [7]:
clusters = []

for component in components:
    if len(component) > 1:
        for i in range(len(component)-1, -1, -1):
            new_row = {'source': component[i],
                       'target': component[i-1]}
            clusters.append(new_row)

In [8]:
output = pd.DataFrame(columns=('source', 'target'))
output = pd.DataFrame(clusters)
output.to_csv('dados/nets.csv', index=False)